# LEGO Detection - Faster R-CNN

### Libraries

In [ ]:
# import libraries

import os
import cv2
import torch
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# from google.colab import drive # running on google colab
from xml.etree import ElementTree as ET
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm import tqdm

!pip install torchmetrics
from torchmetrics.detection import MeanAveragePrecision

### Data preparation

In [ ]:
# possible strategies

cut_remaining = 300
apply_augmentation = False

In [ ]:
# mount drive on colab notebook

# drive.mount('/content/drive') # running on google colab

In [ ]:
# unzip data files

# !unzip "/content/drive/MyDrive/02 - tagged1.zip" -d "/content" # running on google colab

In [ ]:
# major variables

photos_dir = 'content/photos' # running locally
renders_dir = 'content/renders' # running locally

# photos_dir = '/content/photos' # running on google colab
# renders_dir = '/content/renders' # running on google colab

In [ ]:
def load_data(data_dir):
    '''
    Returns a list of images and labels for each image
    '''
    image_paths = []
    num_legos = []
    for subdir, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.jpg'):
                n = int(subdir.split(os.sep)[-1])
                image_paths.append(os.path.join(subdir, file))
                num_legos.append(n)
    combined = list(zip(image_paths, num_legos))
    combined.sort()
    image_paths, num_legos = zip(*combined)
    image_paths = np.asarray(image_paths)
    num_legos = torch.Tensor(num_legos).to(torch.int64)
    return image_paths, num_legos

In [ ]:
def parse_xml(xml_file):
    '''
    Read the xml file and return the bounding box coordinates
    '''
    tree = ET.parse(xml_file)
    root = tree.getroot()
    bounding_boxes = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        bounding_boxes.append([xmin, ymin, xmax, ymax])
    return bounding_boxes

In [ ]:
def parse_all_xml(image_paths):
    '''
    Parse all XML files corresponding to the given image paths.
    '''
    bounding_boxes = []
    for img_path in image_paths:
        xml_path = img_path.replace('.jpg', '.xml')
        bounding_boxes.append(parse_xml(xml_path))
    return bounding_boxes

In [ ]:
# load data

image_paths, num_legos = load_data(photos_dir)

In [ ]:
# parse bounding boxes for all images

bounding_boxes = parse_all_xml(image_paths)

In [ ]:
# class distribution in overall data

plt.hist(num_legos, bins=range(1, max(num_legos)), align='left', rwidth=0.8)
plt.xlabel('Number of LEGOs')
plt.ylabel('Frequency')
plt.title('LEGO Distribution')
plt.show()

In [ ]:
# work with defined train test split

train_test_split = np.genfromtxt('content/train_test_split.csv', delimiter=',', dtype=None, encoding=None) # running locally
# train_test_split = np.genfromtxt('/content/drive/MyDrive/train_test_split.csv', delimiter=',', dtype=None, encoding=None) # running on google colab

train_test_ids = {
    'train': [],
    'test': []
}
for index, row in enumerate(train_test_split):
    if row[1] == '1':
      train_test_ids['test'].append(index - 1)
    elif row[1] == '0':
      train_test_ids['train'].append(index - 1)

len(train_test_ids['train']), len(train_test_ids['test'])

In [ ]:
# class distribution in training data

num_legos_train = num_legos[train_test_ids['train']]
plt.hist(num_legos_train, bins=range(1, max(num_legos_train)), align='left', rwidth=0.8)
plt.xlabel('Number of LEGOs')
plt.ylabel('Frequency')
plt.title('LEGO Training Distribution')
plt.show()

In [ ]:
# validation and test sets

num_legos_test = num_legos[train_test_ids['test']].numpy()
test_ids = train_test_ids['test']
test_distribution = Counter(num_legos_test)
set1_ids, set2_ids = [], []

for cls, count in test_distribution.items():
    cls_ids = [test_ids[i] for i in range(len(test_ids)) if num_legos_test[i] == cls]
    half_count = count // 2
    remainder = count % 2
    set1_ids.extend(cls_ids[:half_count])
    set2_ids.extend(cls_ids[half_count:half_count*2])
    if remainder:
        if len(set1_ids) <= len(set2_ids):
            set1_ids.append(cls_ids[-1])
        else:
            set2_ids.append(cls_ids[-1])

train_test_ids['test'] = np.array(set1_ids)
train_test_ids['valid'] = np.array(set2_ids)

In [ ]:
# class distribution in validation and test data

num_legos_valid = num_legos[train_test_ids['valid']].numpy()
num_legos_test = num_legos[train_test_ids['test']].numpy()

valid_distribution = Counter(num_legos_valid)
test_distribution = Counter(num_legos_test)
    
print("Valid length:", len(train_test_ids['valid']))
print("Test length:", len(train_test_ids['test']))
    
print("\nValid class distribution:")
for cls, count in valid_distribution.items():
    print(f"Class {cls}: {count}")
    
print("\nTest class distribution:")
for cls, count in test_distribution.items():
    print(f"Class {cls}: {count}")

In [ ]:
# undersampling of larger classes in training data

indices1 = []
indices2 = []

for i in train_test_ids['train']:
    if num_legos[i] == 1:
        indices1.append(i)
    elif num_legos[i] == 2:
        indices2.append(i)
        
np.random.shuffle(indices1, )
np.random.shuffle(indices2, )

leftovers1 = indices1[cut_remaining:]
leftovers2 = indices2[cut_remaining:]

for i in leftovers1:
    train_test_ids['train'].remove(i)
for i in leftovers2:
    train_test_ids['train'].remove(i)

num_legos_train = num_legos[train_test_ids['train']]
plt.hist(num_legos_train, bins=range(1, max(num_legos_train)), align='left', rwidth=0.8)
plt.xlabel('Number of LEGOs')
plt.ylabel('Frequency')
plt.title('LEGO Training Distribution (Undersampling)')
plt.show()

In [ ]:
# bounding boxes for train, valid, and test sets

train_boxes = [bounding_boxes[i] for i in train_test_ids['train']]
valid_boxes = [bounding_boxes[i] for i in train_test_ids['valid']]
test_boxes = [bounding_boxes[i] for i in train_test_ids['test']]

In [ ]:
class LegosDataset(Dataset):
    '''
    Dataset class for the legos dataset
    '''
    def __init__(self, images_filenames, num_legos, bounding_boxes, transforms=[], augmented=[]):
        self.images_filenames = images_filenames
        self.labels = num_legos
        self.bounding_boxes = bounding_boxes
        self.transforms = transforms
        self.augmented = augmented

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, id):
        image_filename = self.images_filenames[id]
        label = self.labels[id]
        bounding_boxes = self.bounding_boxes[id]
        image = cv2.imread(image_filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_height, original_width = image.shape[:2]
        transformation = self.transforms[self.augmented[id]]
        image = transformation(image)
        scale_w = 640.0 / original_width
        scale_h = 640.0 / original_height
        scaled_boxes = []
        for box in bounding_boxes:
            x_min, y_min, x_max, y_max = box
            x_min = int(x_min * scale_w)
            y_min = int(y_min * scale_h)
            x_max = int(x_max * scale_w)
            y_max = int(y_max * scale_h)
            scaled_boxes.append([x_min, y_min, x_max, y_max])
        target = {
            'boxes': torch.tensor(scaled_boxes, dtype=torch.float32),
            'labels': torch.ones((label,), dtype=torch.int64)
        }
        return image, target

In [ ]:
# train, valid and test sets

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])

augment = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.5),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor()
])

train_dataset = LegosDataset(image_paths[train_test_ids['train']], num_legos[train_test_ids['train']], train_boxes, 
                             transforms=[transform, augment], augmented=[0]*len(image_paths[train_test_ids['train']]))

valid_dataset = LegosDataset(image_paths[train_test_ids['valid']], num_legos[train_test_ids['valid']], valid_boxes, 
                             transforms=[transform], augmented=[0]*len(image_paths[train_test_ids['valid']]))

test_dataset = LegosDataset(image_paths[train_test_ids['test']], num_legos[train_test_ids['test']], test_boxes, 
                            transforms=[transform], augmented=[0]*len(image_paths[train_test_ids['test']]))

In [ ]:
def generate_data(image_paths, num_legos, bounding_boxes, copies=5):
    '''
    Generate more data by copying images with more than 6 legos
    '''
    new_image_paths = []
    new_num_legos = []
    new_bounding_boxes = []
    for id in range(len(image_paths)):
        if num_legos[id] >= 6:
            for _ in range(copies):
                new_image_paths.append(image_paths[id])
                new_num_legos.append(num_legos[id])
                new_bounding_boxes.append(bounding_boxes[id])
    return new_image_paths, new_num_legos, new_bounding_boxes

In [ ]:
# oversampling of smaller classes in training data - augmentation

if apply_augmentation:
    new_image_paths, new_num_legos, new_bounding_boxes = generate_data(
                                                            image_paths[train_test_ids['train']], 
                                                            num_legos[train_test_ids['train']], 
                                                            train_boxes
                                                        )
    for img, lbl, bbox in zip(new_image_paths, new_num_legos, new_bounding_boxes):
        train_dataset.images_filenames = np.append(train_dataset.images_filenames, img)
        train_dataset.labels = torch.cat((train_dataset.labels, torch.tensor([lbl], dtype=torch.int64)))
        train_dataset.bounding_boxes.append(bbox)
    train_dataset.augmented.extend([1] * len(new_image_paths))
    num_legos_train = train_dataset.labels
    plt.hist(num_legos_train, bins=range(1, max(num_legos_train)), align='left', rwidth=0.8)
    plt.xlabel('Number of LEGOs')
    plt.ylabel('Frequency')
    plt.title('LEGO Training Distribution (Oversampling)')
    plt.show()

In [ ]:
# dataloaders

batch_size = 8
num_workers = 0 # running locally
# num_workers = 2 # running on google colab

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

len(train_dataset), len(valid_dataset), len(test_dataset)

### Model definition

In [ ]:
# get cpu or gpu device for training

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# faster r-cnn model

faster_rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

in_features = faster_rcnn.roi_heads.box_predictor.cls_score.in_features
num_classes = 2
faster_rcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
# put model in device

model = faster_rcnn.to(device)

model

### Model training

In [ ]:
# possible strategies

train_only_head = True
train_head_and_rpn = True
train_all = True

In [ ]:
def apply_nms(predictions, iou_threshold=0.3):
    """
    Apply non-maximum suppression to predictions
    """
    boxes = predictions['boxes']
    scores = predictions['scores']
    labels = predictions['labels']
    keep = torchvision.ops.nms(boxes, scores, iou_threshold=iou_threshold)
    boxes = boxes[keep]
    scores = scores[keep]
    labels = labels[keep]
    return boxes, scores, labels

In [ ]:
def compute_map(preds, targets):
    """
    Calculates the Mean Average Precision (mAP) for object detection predictions
    """
    metric = MeanAveragePrecision(box_format='xyxy', iou_type='bbox')
    metric.update(preds=preds, target=targets)
    values = metric.compute()
    return values['map'].item()

In [ ]:
def epoch_iter(dataloader, model, optimizer=None, is_train=True, grad_accum_steps=2):
    '''
    Perform one epoch of training/validation/testing
    '''
    if is_train:
        assert optimizer is not None, "When training, please provide an optimizer"
    num_batches = len(dataloader)
    model.train() if is_train else model.eval()
    total_loss = 0.0
    total_loss_classifier = 0.0
    total_loss_box_reg = 0.0
    total_loss_objectness = 0.0
    total_loss_rpn_box_reg = 0.0
    total_map = 0.0
    with torch.set_grad_enabled(is_train):
        for batch, (images, targets) in enumerate(tqdm(dataloader)):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            if is_train:
                loss_data = model(images, targets)
                loss_classifier = loss_data['loss_classifier']
                total_loss_classifier += loss_classifier.item()
                loss_box_reg = loss_data['loss_box_reg']
                total_loss_box_reg += loss_box_reg.item()
                loss_objectness = loss_data['loss_objectness']
                total_loss_objectness += loss_objectness.item()
                loss_rpn_box_reg = loss_data['loss_rpn_box_reg']
                total_loss_rpn_box_reg += loss_rpn_box_reg.item()
                losses = (loss_classifier + loss_box_reg + loss_objectness + loss_rpn_box_reg) / grad_accum_steps
                optimizer.zero_grad()
                losses.backward()
                if (batch + 1) % grad_accum_steps == 0:
                    optimizer.step()
                total_loss += losses.item() * grad_accum_steps
            else:
                with torch.no_grad():
                    predictions = model(images)
                    for preds in predictions:
                        preds['boxes'], preds['scores'], preds['labels'] = apply_nms(preds)
                    map_score = compute_map(predictions, targets)
                    total_map += map_score
    avg_loss = total_loss / num_batches if is_train else None
    avg_loss_classifier = total_loss_classifier / num_batches if is_train else None
    avg_loss_box_reg = total_loss_box_reg / num_batches if is_train else None
    avg_loss_objectness = total_loss_objectness / num_batches if is_train else None
    avg_loss_rpn_box_reg = total_loss_rpn_box_reg / num_batches if is_train else None
    avg_map = total_map / num_batches if not is_train else None
    return {
        'avg_loss': avg_loss,
        'avg_loss_classifier': avg_loss_classifier,
        'avg_loss_box_reg': avg_loss_box_reg,
        'avg_loss_objectness': avg_loss_objectness,
        'avg_loss_rpn_box_reg': avg_loss_rpn_box_reg,
        'avg_map': avg_map
    }

In [ ]:
def train(model, model_name, num_epochs, train_dataloader, validation_dataloader, optimizer, train_history=None, val_history=None):
    '''
    Train the model
    '''
    if train_history is None:
        train_history = {
            'loss': [],
            'loss_classifier': [],
            'loss_box_reg': [],
            'loss_objectness': [],
            'loss_rpn_box_reg': []
        }
    if val_history is None:
        val_history = {'map': []}
    best_val_map = -np.inf
    print("Start training...")
    for t in range(num_epochs):
        print(f"\nEpoch {t+1}/{num_epochs}")
        train_metrics = epoch_iter(train_dataloader, model, optimizer, is_train=True)
        train_loss = train_metrics['avg_loss']
        train_loss_classifier = train_metrics['avg_loss_classifier']
        train_loss_box_reg = train_metrics['avg_loss_box_reg']
        train_loss_objectness = train_metrics['avg_loss_objectness']
        train_loss_rpn_box_reg = train_metrics['avg_loss_rpn_box_reg']
        print(f"Train loss: {train_loss:.3f}")
        print(f"  Classifier loss: {train_loss_classifier:.3f}")
        print(f"  Box regression loss: {train_loss_box_reg:.3f}")
        print(f"  Objectness loss: {train_loss_objectness:.3f}")
        print(f"  RPN box regression loss: {train_loss_rpn_box_reg:.3f}")
        val_metrics = epoch_iter(validation_dataloader, model, is_train=False)
        val_map = val_metrics['avg_map']
        print(f"Validation mAP: {val_map:.3f}")
        if val_map > best_val_map:
            best_val_map = val_map
            save_dict = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': t
            }
            torch.save(save_dict, model_name + '_best_model.pth')
        save_dict = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': t
        }
        torch.save(save_dict, model_name + '_latest_model.pth')
        train_history['loss'].append(train_loss)
        train_history['loss_classifier'].append(train_loss_classifier)
        train_history['loss_box_reg'].append(train_loss_box_reg)
        train_history['loss_objectness'].append(train_loss_objectness)
        train_history['loss_rpn_box_reg'].append(train_loss_rpn_box_reg)
        val_history['map'].append(val_map)
    print("Finished")
    return train_history, val_history

In [ ]:
# training and validation history

train_history = {
    'loss': [],
    'loss_classifier': [],
    'loss_box_reg': [],
    'loss_objectness': [],
    'loss_rpn_box_reg': []
}
val_history = {'map': []}

In [ ]:
# model layers

for param in model.parameters():
    param.requires_grad = False
    
head_layers = ['roi_heads.box_predictor']
rpn_layers = [layer[0] for layer in list(model.named_parameters()) if 'rpn' in layer[0]]
backbone_layers = [layer[0] for layer in list(model.named_parameters()) if 'backbone' in layer[0]]

In [ ]:
def unfreeze_layers(model, layers_to_unfreeze):
    '''
    Unfreeze the specified layers in the model
    '''
    for name, param in model.named_parameters():
        if any(layer in name for layer in layers_to_unfreeze):
            param.requires_grad = True

In [ ]:
# train head

if train_only_head:
    unfreeze_layers(model, head_layers)
    optimizer = torch.optim.Adam(model.roi_heads.box_predictor.parameters(), lr=1e-3)
    num_epochs = 7
    train_history, val_history = train(
                                    model,
                                    'lego_detector',
                                    num_epochs,
                                    train_dataloader,
                                    valid_dataloader,
                                    optimizer,
                                    train_history,
                                    val_history
                                )

In [ ]:
# train head + RPN

if train_head_and_rpn:
    model = faster_rcnn.to(device)
    checkpoint = torch.load('lego_detector_latest_model.pth')
    model.load_state_dict(checkpoint['model'])
    del checkpoint
    unfreeze_layers(model, head_layers)
    unfreeze_layers(model, rpn_layers)
    optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, momentum=0.9, weight_decay=0.0005)
    num_epochs = 3
    train_history, val_history = train(
                                model,
                                'lego_detector',
                                num_epochs,
                                train_dataloader,
                                valid_dataloader,
                                optimizer,
                                train_history,
                                val_history
                                )

In [ ]:
# train head + RPN + backbone

if train_all:
    torch.cuda.empty_cache()
    model = faster_rcnn.to(device)
    checkpoint = torch.load('lego_detector_best_model.pth')
    model.load_state_dict(checkpoint['model'])
    del checkpoint
    unfreeze_layers(model, head_layers)
    unfreeze_layers(model, rpn_layers)
    unfreeze_layers(model, backbone_layers)
    for param in model.parameters():
        param.requires_grad = True
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=0.0005)
    num_epochs = 20
    train_history, val_history = train(
                                model,
                                'lego_detector',
                                num_epochs,
                                train_dataloader,
                                valid_dataloader,
                                optimizer,
                                train_history,
                                val_history
                                )

### Training evolution analysis

In [ ]:
def plotTrainingLoss(train_history):
    '''
    Plot the training loss history of the model
    '''
    plt.figure(figsize=(10, 4))
    plt.plot(train_history['loss'], label='Training Loss')
    plt.plot(train_history['loss_classifier'], label='Classifier Loss')
    plt.plot(train_history['loss_box_reg'], label='Box Regression Loss')
    plt.plot(train_history['loss_objectness'], label='Objectness Loss')
    plt.plot(train_history['loss_rpn_box_reg'], label='RPN Box Regression Loss')
    plt.title('Training Loss History')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
def plotValidationMAP(val_history):
    '''
    Plot the validation mAP history of the model
    '''
    plt.figure(figsize=(10, 4))
    plt.plot(val_history['map'], label='Validation mAP')
    plt.title('Validation mAP History')
    plt.xlabel('Epochs')
    plt.ylabel('mAP')
    plt.legend()
    plt.show()

In [ ]:
# visualize training history

plotTrainingLoss(train_history)

In [ ]:
# visualize validation history

plotValidationMAP(val_history)

### Model testing

In [ ]:
# load best model

model = faster_rcnn.to(device)
checkpoint = torch.load('lego_detector_best_model.pth')
model.load_state_dict(checkpoint['model'])

In [ ]:
# evaluate model on test data

test_metrics = epoch_iter(test_dataloader, model, is_train=False)
test_map = test_metrics['avg_map']
print(f"\nTest mAP: {test_map:.3f}")

In [ ]:
def show_predictions(model, dataloader, num_images=10):
    '''
    Display ground truth vs prediction in images
    '''
    model.eval()
    image_count = 0
    for batch, (images, targets) in enumerate(dataloader):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.no_grad():
            predictions = model(images)
            for i in range(len(images)):
                fig, axes = plt.subplots(1, 2, figsize=(8, 4))
                axes[0].imshow(transforms.ToPILImage()(images[i]))
                axes[0].set_title('True')
                axes[0].axis('off')
                for box in targets[i]['boxes']:
                    box = box.cpu().numpy()
                    rect = patches.Rectangle(
                        (box[0], box[1]), 
                        box[2] - box[0], 
                        box[3] - box[1], 
                        linewidth=2, 
                        edgecolor='g', 
                        facecolor='none'
                    )
                    axes[0].add_patch(rect)
                axes[1].imshow(transforms.ToPILImage()(images[i]))
                axes[1].set_title('Predicted')
                axes[1].axis('off')
                boxes, scores, labels = apply_nms(predictions[i])
                for box, score, label in zip(boxes, scores, labels):
                    if score > 0.5:
                        box = box.cpu().numpy()
                        rect = patches.Rectangle(
                            (box[0], box[1]), 
                            box[2] - box[0], 
                            box[3] - box[1], 
                            linewidth=2, 
                            edgecolor='b', 
                            facecolor='none'
                        )
                        axes[1].add_patch(rect)
                        axes[1].text(box[0], box[1], f'{score:.2f}', bbox=dict(facecolor='yellow', alpha=0.5))
                plt.show()
                image_count += 1
                if image_count >= num_images:
                    break
        if image_count >= num_images:
            break

In [ ]:
# view predictions

show_predictions(model, test_dataloader)